## 4. Measuring Phenotypic Activity

In this notebook, we measure the **phenotypic activity** of compounds by comparing them against the negative control. To do this, we focus on the treatment-specific clusters identified earlier, which allow us to capture the **heterogeneous effects** that each treatment produces across different subpopulations of cells. We also make use of the *on* and *off* signatures.

Our approach is based on the **Earth Mover’s Distance (EMD)**, a distance metric provides an understanding of how much “work” is needed to transform one distribution (cells treated with a compound) into another (e.g., the healthy cell state as a control).

We calculate EMD on two sets of features: those defining the *on-signature* and those defining the *off-signature*. This can be interpreted as follows:

* **On-signature scores:**

  * High EMD → the treatment/perturbation produces strong differences in morphology features associated with the targeted cell state.
  * Low EMD → the targeted features resemble the controls, suggesting potential evidence of reversal.

* **Off-signature scores:**

  * High EMD → strong off-target effects, since these features are unrelated to the specific cell state.
  * Low EMD → minimal off-target effects, indicating the compound acts more selectively.

In this notebook, we provide an example from the CFReT dataset to investigate the reversal of cardiac fibrosis. Our goal is to determine whether the treated group (drug_x) contains subpopulations that resemble the control group. Clusters with low EMD scores for both the “on” and “off” signatures suggest that a potential reversal is occurring within specific subpopulations of cells. In contrast, clusters with high “off” scores and low “on” scores indicate that, while the targeted morphological signatures appear to resemble a healthier state, many non-targeted morphological features remain altered and are not associated with a specific cellular state.

In [1]:
import sys
import pathlib

import polars as pl

sys.path.append("../../")
from utils.metrics import measure_phenotypic_activity
from utils import io_utils

helper functions

setting import and output paths

In [8]:
# setting directories
data_dir = pathlib.Path("../0.download-data/data/sc-profiles").resolve(strict=True)
results_dir = pathlib.Path("./results").resolve(strict=True)
signatures_dir = (results_dir / "signature_results").resolve(strict=True)
cluster_labels_dir = (results_dir / "cluster-labels").resolve(strict=True)

# setting cpjump1 profile paths, signatures, cluster labels
cpjump1_negcon_profiles_path = (
    (data_dir / "cpjump1" / "negcon").resolve(strict=True).glob("*.parquet")
)
cpjump1_trt_profiles_path = (
    data_dir / "cpjump1" / "trt-profiles" / "cpjump1_crispr_trt_profiles.parquet"
).resolve(strict=True)
cpjump1_signatures_path = (
    results_dir / "signature_results" / "ks_test_cpjump1_consensus_signatures.json"
).resolve(strict=True)

# setting cfret1 profile paths, signatures, cluster labels
cfret_profiles_path = (
    data_dir / "cfret" / "localhost230405150001_sc_feature_selected.parquet"
).resolve(strict=True)
cfret_signatures_path = (signatures_dir / "ks_test_cfret_signatures.json").resolve(
    strict=True
)
cfret_cluster_labels_path = (
    cluster_labels_dir / "cfret_heterogenic_clusters_results.json"
).resolve(strict=True)

# setting mitocheck profile paths, signatures, cluster labels
# mitocheck_negcon_profiles_path = (
#     data_dir / "mitocheck" / "negcon_mitocheck_cp_profiles.parquet"
# ).resolve(strict=True)
# mitocheck_trt_profiles_path = (
#     data_dir / "mitocheck" / "treated_mitocheck_cp_profiles.parquet"
# ).resolve(strict=True)
# mitocheck_signatures_path = (
#     signatures_dir / "ks_test_mitocheck_signatures.json"
# ).resolve(strict=True)
# mitocheck_cluster_labels_path = (
#     cluster_labels_dir / "mitocheck_cluster_results.pkl"
# ).resolve(strict=True)

# setting output paths
metric_out_dir = (results_dir / "metrics").resolve()
metric_out_dir.mkdir(exist_ok=True)

In [15]:
# load cfret profiles
cfret_profiles = pl.read_parquet(cfret_profiles_path)

# load mitocheck
# mitocheck_negcon_profiles = pl.read_parquet(mitocheck_negcon_profiles_path)
# mitocheck_trt_profiles = pl.read_parquet(mitocheck_trt_profiles_path)

# load signatures (pickle files)
# mitocheck_sigs = io_utils.load_configs(mitocheck_signatures_path)
cfret_sigs = io_utils.load_configs(cfret_signatures_path)

# load cluster labels (pickle files)
# mitocheck_cluster_labels = io_utils.load_configs(mitocheck_cluster_labels_path)
cfret_cluster_labels = io_utils.load_configs(cfret_cluster_labels_path)

In [16]:
# adding cluster labels to the profiles
# mitocheck_negcon_profiles = mitocheck_negcon_profiles.with_columns(
#     pl.lit(0).alias("Metadata_cluster")
# )
# mitocheck_trt_profiles = mitocheck_trt_profiles.with_columns(
#     pl.Series("Metadata_cluster", mitocheck_cluster_labels["cluster_labels"])
# )


cfret_profiles = cfret_profiles.with_columns(
    pl.Series("Metadata_cluster", cfret_cluster_labels["cfret_cluster_results"])
)
negcon_cfret_profiles = cfret_profiles.filter(pl.col("Metadata_treatment") == "DMSO")
treated_cfret_profiles = cfret_profiles.filter(pl.col("Metadata_treatment") != "DMSO")

## Measuring phenotypic activity

In [17]:
cfret_phenotypic_activity = measure_phenotypic_activity(
    ref_profile=negcon_cfret_profiles,
    exp_profiles=treated_cfret_profiles,
    on_signature=cfret_sigs["cfret_negcon_TGFRi_poscon"]["signatures"]["on"],
    off_signature=cfret_sigs["cfret_negcon_TGFRi_poscon"]["signatures"]["off"],
    method="emd",
    treatment_col="Metadata_treatment",
    emd_dist_matrix_method="euclidean",
)

In [22]:
# save phenotypic activity scores
cfret_phenotypic_activity.sort(
    ["on_dist", "off_dist"], descending=[False, False]
).write_csv(metric_out_dir / "cfret_phenotypic_activity_scores.csv")

In [20]:
# sort based on on_dist and low_dist ascending order
cfret_phenotypic_activity = cfret_phenotypic_activity.sort(
    ["on_dist", "off_dist"], descending=[False, False]
)
# --- IGNORE ---

In [21]:
cfret_phenotypic_activity

ref_cluster,treatment,exp_cluster,on_dist,off_dist
str,str,str,f64,f64
"""5""","""TGFRi""","""0""",24.649985,7.201024
"""0""","""TGFRi""","""0""",24.69521,7.184525
"""2""","""TGFRi""","""0""",24.729591,7.206523
"""5""","""TGFRi""","""1""",24.776689,7.196115
"""0""","""TGFRi""","""1""",24.780459,7.189827
…,…,…,…,…
"""4""","""drug_x""","""3""",27.957917,7.453054
"""1""","""drug_x""","""2""",27.976122,7.497608
"""4""","""drug_x""","""5""",27.977291,7.511904
